In [1]:
import numpy as np
from tqdm import tqdm
import math

### Let's get started with some data preparations

In [2]:
MAX_VALUE = 2**32

In [3]:
with open("output.bin", "wb") as file:
    big_ints = np.random.randint(
                    low=0,
                    high=MAX_VALUE,
                    size=5000,
                    dtype=np.uint32,
                )
    big_ints = big_ints.newbyteorder()
    assert (
        big_ints.dtype.byteorder == ">"
        ), "Result array somehow is not big endian"
    file.write(big_ints.tobytes())

In [4]:
with open("output.bin", "rb") as file:
        data = np.fromfile(file, dtype=">u4", count=5000)

In [5]:
def get_prime_factors_amount(number):
    count = 0

    # Check if 2 is a prime factor
    while number % 2 == 0:
        count += 1
        number //= 2

    # Check for other prime factors
    for i in range(3, int(math.sqrt(number)) + 1, 2):
        while number % i == 0:
            count += 1
            number //= i

    # If the given number is still greater than 1, it is a prime factor
    if number > 1:
        count += 1

    return count

### Simple sequentual algorithm

In [6]:
def dummy_calculate(data):
    return sum(get_prime_factors_amount(number) for number in tqdm(data))

In [7]:
%%timeit
print(f"Amount of prime numbers is: {dummy_calculate(data)}")

100%|██████████| 5000/5000 [00:16<00:00, 308.51it/s]


Amount of prime numbers is: 20857


100%|██████████| 5000/5000 [00:16<00:00, 309.57it/s]


Amount of prime numbers is: 20857


100%|██████████| 5000/5000 [00:16<00:00, 304.70it/s]


...

Amount of prime numbers is: 20857


100%|██████████| 5000/5000 [00:16<00:00, 310.93it/s]


Amount of prime numbers is: 20857


100%|██████████| 5000/5000 [00:16<00:00, 308.38it/s]

Amount of prime numbers is: 20857
16.2 s ± 101 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Multiprocessing algorithm

In [8]:
from multiprocessing import Pool
from multiprocessing import cpu_count

In [9]:
device_amount = cpu_count()
print(f"Cpu amount is {device_amount}")

Cpu amount is 4


In [10]:
def mp_calculate(data):
    with Pool(device_amount) as pool:
        results = tqdm(
            pool.imap_unordered(
                get_prime_factors_amount,
                data,
            ),
            total=len(data),
        )

        return sum(results)

In [11]:
%%timeit
print(f"Amount of prime numbers is: {mp_calculate(data)}")

100%|██████████| 5000/5000 [00:10<00:00, 496.10it/s]


Amount of prime numbers is: 20857


100%|██████████| 5000/5000 [00:10<00:00, 494.53it/s]

Amount of prime numbers is: 20857



100%|██████████| 5000/5000 [00:09<00:00, 504.64it/s]

...

Amount of prime numbers is: 20857



100%|██████████| 5000/5000 [00:09<00:00, 503.12it/s]

Amount of prime numbers is: 20857



100%|██████████| 5000/5000 [00:10<00:00, 493.72it/s]


Amount of prime numbers is: 20857
10.1 s ± 96.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Implementation using PySpark

In [12]:
from pyspark.sql import SparkSession
from operator import add

spark = SparkSession.builder.getOrCreate()

23/06/07 19:46:09 WARN Utils: Your hostname, codespaces-3c66f5 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/06/07 19:46:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/07 19:46:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
rdd = spark.sparkContext.parallelize(data, device_amount)

In [14]:
def spark_calculate(rdd):
    whole_amount = rdd.map(get_prime_factors_amount).reduce(add)
    return whole_amount

In [16]:
%%timeit
print(f"Amount of prime numbers is: {spark_calculate(rdd)}")

Amount of prime numbers is: 20857


Amount of prime numbers is: 20857


Amount of prime numbers is: 20857


Amount of prime numbers is: 20857


Amount of prime numbers is: 20857


Amount of prime numbers is: 20857


Amount of prime numbers is: 20857


Amount of prime numbers is: 20857
10.3 s ± 68.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Go for go solution!

Source code may be found in sub-directory named `goroutines`

In [17]:
!time go run main.go -input_file output.bin

{"level":"info","time":"2023-06-07T19:49:09Z","message":"Goes with 4 cpus on board."}
{"level":"info","time":"2023-06-07T19:49:09Z","message":"Done! Amount of prime numbers for ints in output.bin: 20857"}

real	0m8.239s
user	0m18.332s
sys	0m2.645s


`real` - real program execution time, `user+sys` – CPU program execution time (processor time in kernel and user mode).

### Summary
TL;DR: Golang goes brrrrr)))
Below is a list of execution times from fastest to slowest:
1. Golang
2. PySpark
3. Multiprocessing way
4. Sequential way